In [1]:
#@title 檢視 GPU
! nvidia-smi

Fri Mar 11 14:37:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title 安裝 colorama, ffmpeg

! pip install colorama
! pip install ffmpeg-python

In [3]:
#@title 安裝 DeepFaceLab
!git clone https://github.com/iperov/DeepFaceLab.git

Cloning into 'DeepFaceLab'...
remote: Enumerating objects: 8010, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 8010 (delta 146), reused 179 (delta 118), pack-reused 7751
Receiving objects: 100% (8010/8010), 823.27 MiB | 38.65 MiB/s, done.
Resolving deltas: 100% (5143/5143), done.
Checking out files: 100% (211/211), done.


In [4]:
#@title 建立 workspace
!mkdir -p /content/workspace; mkdir -p /content/workspace/data_src; mkdir -p /content/workspace/data_src/aligned; mkdir -p /content/workspace/data_dst; mkdir -p /content/workspace/data_dst/aligned; mkdir -p /content/workspace/model  
! ls workspace

data_dst  data_src  model


In [5]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 用youtube-dl下載影片
! curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
! chmod a+rx /usr/local/bin/youtube-dl
src_youtube = 'https://www.youtube.com/watch?v=EKS6mfLtR2s' #@param {type:"string"}
dst_youtube = 'https://www.youtube.com/watch?v=LnKlifb_dQ0'#@param {type:"string"}
src_yt_cmd =  f'youtube-dl -f mp4 {src_youtube}  --output /content/workspace/data_src.mp4'
dst_yt_cmd =  f'youtube-dl -f mp4 {dst_youtube}  --output /content/workspace/data_dst.mp4'
! $src_yt_cmd
! $dst_yt_cmd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0     44      0 --:--:-- --:--:-- --:--:--    44
100     3  100     3    0     0     38      0 --:--:-- --:--:-- --:--:--    38
100   645  100   645    0     0   5466      0 --:--:-- --:--:-- --:--:--  5466
100 1794k  100 1794k    0     0  5789k      0 --:--:-- --:--:-- --:--:-- 30.9M
[youtube] EKS6mfLtR2s: Downloading webpage
[download] Destination: /content/workspace/data_src.mp4
[download]   0.4% of 61.07MiB at 48.91KiB/s ETA 21:13
ERROR: Interrupted by user


In [6]:
#@title 從 Google Drive 複製影片到workspace

src_mp4 = '/content/drive/MyDrive/aidataset/deepfacelab/data_src.mp4' #@param {type:"string"}
dst_mp4 = '/content/drive/MyDrive/aidataset/deepfacelab/data_dst.mp4' #@param {type:"string"}
src_cmd=f"cp {src_mp4} /content/workspace/data_src.mp4"  
dst_cmd=f"cp {dst_mp4} /content/workspace/data_dst.mp4" 
! $src_cmd
! $dst_cmd

In [9]:
#@title 防止 Google Colab 斷開連線

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [7]:
#@title 從 data_src 影片擷取人物圖片
# 開始擷取 src
%cd /content/DeepFaceLab/
! python main.py videoed extract-video --input-file /content/workspace/data_src.mp4 --output-dir /content/workspace/data_src/

/content/DeepFaceLab
[0] Enter FPS ( ?:help ) : 
0
[png] Output image format ( png/jpg ?:help ) : 
png
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libv

In [8]:
#@title 從 data_dst 影片擷取人物圖片
# 開始擷取 dst
%cd /content/DeepFaceLab/
! python main.py videoed extract-video --input-file /content/workspace/data_dst.mp4 --output-dir /content/workspace/data_dst/

/content/DeepFaceLab
[0] Enter FPS ( ?:help ) : 
0
[png] Output image format ( png/jpg ?:help ) : 
png
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libv

In [10]:
#@title data_src 影片降噪 (選擇性)
%cd /content/DeepFaceLab/
! python  main.py videoed denoise-image-sequence --input-dir /content/workspace/data_src 

/content/DeepFaceLab
[7] Denoise factor? ( 1-20 ) : 
7
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable

In [11]:
#@title data_dst 影片降噪 (選擇性)
%cd /content/DeepFaceLab/
! python  main.py videoed denoise-image-sequence --input-dir /content/workspace/data_dst 

/content/DeepFaceLab
[7] Denoise factor? ( 1-20 ) : 
7
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable

In [12]:
#@title 校正 data_src 影片
%cd /content/DeepFaceLab/
! python main.py extract --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --detector s3fd

/content/DeepFaceLab

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla T4

[0] Which GPU indexes to choose? : 0
0

[wf] Face type ( f/wf/head ?:help ) : 
wf
[0] Max number of faces from image ( ?:help ) : 
0
[512] Image size ( 256-2048 ?:help ) : 
512
[90] Jpeg quality ( 1-100 ?:help ) : 
90
[n] Write debug images to aligned_debug? ( y/n ) : y
Extracting faces...
Running on Tesla T4
100% 1650/1650 [09:20<00:00,  2.95it/s]
-------------------------
Images found:        1650
Faces detected:      1650
-------------------------
Done.


In [13]:
#@title 校正 data_dst 影片
%cd /content/DeepFaceLab/
! python main.py extract --input-dir /content/workspace/data_dst --output-dir /content/workspace/data_dst/aligned --detector s3fd

/content/DeepFaceLab

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla T4

[0] Which GPU indexes to choose? : 
0

[wf] Face type ( f/wf/head ?:help ) : 
wf
[0] Max number of faces from image ( ?:help ) : 
0
[512] Image size ( 256-2048 ?:help ) : 
512
[90] Jpeg quality ( 1-100 ?:help ) : 
90
[n] Write debug images to aligned_debug? ( y/n ) : y
Extracting faces...
Running on Tesla T4
100% 1025/1025 [07:34<00:00,  2.26it/s]
-------------------------
Images found:        1025
Faces detected:      1023
-------------------------
Done.


In [14]:
#@title 排序 data_src 影格  (選擇性)
%cd /content/DeepFaceLab/
! python main.py sort --input-dir /content/workspace/data_src/aligned --by hist

/content/DeepFaceLab
Running sort tool.

Sorting by histogram similarity...
Running on 1 threads
Sorting: 100% 1650/1650 [00:10<00:00, 161.16it/s]
Renaming: 100% 1650/1650 [00:00<00:00, 23538.90it/s]
Done.


In [15]:
#@title 排序 data_dst 影格 (選擇性)
%cd /content/DeepFaceLab/
! python main.py sort --input-dir /content/workspace/data_dst/aligned --by hist

/content/DeepFaceLab
Running sort tool.

Sorting by histogram similarity...
Running on 1 threads
Sorting: 100% 1023/1023 [00:05<00:00, 183.20it/s]
Renaming: 100% 1023/1023 [00:00<00:00, 25694.33it/s]
Done.


In [ ]:
#@title 加強 data_src 影格解析度 (選擇性)
%cd /content/DeepFaceLab/
! python  main.py facesettool enhance --input-dir /content/workspace/data_src/aligned

In [ ]:
#@title 加強 data_dst 影格解析度 (選擇性)
%cd /content/DeepFaceLab/
! python  main.py facesettool enhance --input-dir /content/workspace/data_dst/aligned

In [21]:
#@title 打包 data_src 影格
%cd /content/DeepFaceLab/
!python main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset

/content/DeepFaceLab
Performing faceset packing...

Loading samples: 100% 1650/1650 [00:06<00:00, 240.50it/s]
Processing: 100% 1650/1650 [00:00<00:00, 19446.34it/s]
Packing: 100% 1650/1650 [00:00<00:00, 19627.90it/s]
[y] Delete original files? ( y/n ) : n
Done.


In [22]:
#@title 打包 data_dst 影格
%cd /content/DeepFaceLab/
!python main.py util --input-dir /content/workspace/data_dst/aligned  --pack-faceset

/content/DeepFaceLab
Performing faceset packing...

Loading samples: 100% 1023/1023 [00:05<00:00, 185.43it/s]
Processing: 100% 1023/1023 [00:00<00:00, 18527.53it/s]
Packing: 100% 1023/1023 [00:00<00:00, 17538.20it/s]
[y] Delete original files? ( y/n ) : n
Done.


In [ ]:
#@title 壓縮素材
% cd /content/
workspace = 'deepfacelab_ws' #@param {type:"string"}
cmd = f'zip -r {workspace} workspace/*'
! $cmd

In [24]:
#@title 將素材搬移至 Google Drive
workspace_dir = '/content/drive/MyDrive/aidataset/deepfacelab/' #@param {type:"string"}
cmd  = f'cp {workspace}.zip {workspace_dir}'
! $cmd